In [1]:
import os
import numpy as np
import torch

from data_loader.holi_raw_loader import MultiRawDataset

dataset = MultiRawDataset(
    root="/data/test_holoassist",
    label_root="/data/holoassist",
    subset="train",
    max_samples=100,
    transform=None,
    target_transform=None,
)

sample_dict = dataset[0]
for key in sample_dict.keys():
    print(key, sample_dict[key].shape)

/home/jhseon/anaconda3/envs/mnm/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:14: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.1)
  from scipy.sparse import csr_matrix, issparse


rgb (100, 3, 504, 896)
head (100, 4, 4)
hands-left (100, 26, 3)
hands-right (100, 26, 3)
depth (100, 256, 454)
disparity (100, 256, 454)
hands-left-traj (100, 3, 504, 896)
hands-right-traj (100, 3, 504, 896)
raymap (100, 6, 504, 896)


In [2]:
from scipy.spatial.transform import Rotation as R

# 좌표계를 시각화하는 유틸리티 함수 추가
def plot_coordinate_frame(ax, pose, scale=0.1):
    """
    주어진 포즈 행렬로부터 좌표계 프레임을 그리는 함수
    
    Parameters:
    - ax: matplotlib 3D 축 객체
    - pose: 4x4 포즈 행렬
    - scale: 좌표축의 길이 조절 인자
    """
    origin = pose[:3, 3]
    x_axis = origin + scale * pose[:3, 0]
    y_axis = origin + scale * pose[:3, 1]
    z_axis = origin + scale * pose[:3, 2]
    
    # X축 (빨간색)
    ax.plot([origin[0], x_axis[0]], [origin[1], x_axis[1]], [origin[2], x_axis[2]], 'r-', linewidth=2)
    # Y축 (녹색)
    ax.plot([origin[0], y_axis[0]], [origin[1], y_axis[1]], [origin[2], y_axis[2]], 'g-', linewidth=2)
    # Z축 (파란색)
    ax.plot([origin[0], z_axis[0]], [origin[1], z_axis[1]], [origin[2], z_axis[2]], 'b-', linewidth=2)

# --- IMU utilities ---
def read_imu_file(path):
    """
    Reads a sync text with format:
      rel_time  abs_timestamp  x  y  z
    Returns:
      rel_times: (N,) float seconds
      abs_times: (N,) float (nanoseconds)
      vals:      (N,3) float sensor readings
    """
    arr = []
    with open(path) as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.strip().split()
            rel = float(parts[0])
            ts  = float(parts[1])
            vals = list(map(float, parts[2:5]))
            arr.append([rel, ts] + vals)
    data = np.array(arr, dtype=np.float64)
    return data[:,0], data[:,1], data[:,2:5]

def integrate_gyro(rel_times, gyro_vals):
    """
    Integrate gyroscope rates to rotation matrices.
    rel_times: (N,) seconds
    gyro_vals: (N,3) rad/s
    Returns:
      rotations: list of (3,3) rotation matrices, length N
    """
    N = len(rel_times)
    rotations = [np.eye(3, dtype=np.float64)]
    for i in range(1, N):
        dt = rel_times[i] - rel_times[i-1]
        omega = gyro_vals[i] * dt  # small-angle
        dR = R.from_rotvec(omega).as_matrix()
        R_i = rotations[-1] @ dR
        rotations.append(R_i)
    return rotations

def integrate_imu_gyro(rel_times, acc_vals, gyro_vals):
    """
    Double-integrate accelerometer in world frame, using gyro-integrated rotations.
    Returns:
      positions: (N,3)
      rotations: list of (3,3)
    """
    rotations = integrate_gyro(rel_times, gyro_vals)
    g = np.array([0,0,9.81], dtype=np.float64)
    N = len(rel_times)
    v = np.zeros(3, dtype=np.float64)
    p = np.zeros(3, dtype=np.float64)
    positions = np.zeros((N,3), dtype=np.float64)
    positions[0] = p
    for i in range(1, N):
        dt = rel_times[i] - rel_times[i-1]
        R_i = rotations[i]
        acc_world = R_i @ acc_vals[i] - g
        v += acc_world * dt
        p += v * dt
        positions[i] = p
    return positions, rotations

from sklearn.neighbors import NearestNeighbors

def align_to_ref_times(data, data_times, ref_times):
    """Align data to reference timestamps using nearest neighbor."""
    if len(data) == 0:
        # fallback
        return np.zeros((len(ref_times), *data.shape[1:]), dtype=data.dtype)
    nn = NearestNeighbors(n_neighbors=1).fit(data_times.reshape(-1, 1))
    _, indices = nn.kneighbors(ref_times.reshape(-1, 1))
    return data[indices[:,0]]

In [9]:
base_path = os.path.join(dataset.root, dataset.sessions[0], "Export_py")
    
imu_dir = os.path.join(base_path, "IMU")
acc_rel, acc_ns, acc = read_imu_file(os.path.join(imu_dir, "Accelerometer_sync.txt"))
gyro_rel, gyro_ns, gyro = read_imu_file(os.path.join(imu_dir, "Gyroscope_sync.txt"))
imu_positions, imu_rotations = integrate_imu_gyro(acc_rel, acc, gyro)

# Align IMU data to reference times
ref_times = sample_dict['ref_times']
pos_aligned = align_to_ref_times(imu_positions, acc_ns, ref_times)
rot_aligned = align_to_ref_times(np.stack(imu_rotations), acc_ns, ref_times)


In [11]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import numpy as np

# -- Prepare data --
# sample_dict['head']: array of shape (T, 4, 4); sample_dict['rgb']: list or array of shape (T, 3, H, W)
poses = sample_dict['head']
positions = imu_positions #poses[:, :3, 3]    # (T, 3)
rotations = poses[:, :3, :3]   # (T, 3, 3)
images = [frame.transpose(1, 2, 0) for frame in sample_dict['rgb']]


In [13]:
rotations.shape

(100, 3, 3)

In [ ]:

# Forward direction in camera frame is the +Z axis
dirs = rotations @ np.array([0, 0, 1])  # (T, 3)

# -- Set up figure and axes --
fig = plt.figure(figsize=(10, 5))
ax3d = fig.add_subplot(121, projection='3d')
ax_img = fig.add_subplot(122)

# Configure 3D axes limits and labels
padding = 0.1 * (positions.max(axis=0) - positions.min(axis=0))
min_vals = positions.min(axis=0) - padding
max_vals = positions.max(axis=0) + padding
ax3d.set_xlim(min_vals[0], max_vals[0])
ax3d.set_ylim(min_vals[1], max_vals[1])
ax3d.set_zlim(min_vals[2], max_vals[2])
ax3d.set_xlabel('X')
ax3d.set_ylabel('Y')
ax3d.set_zlabel('Z')
ax3d.view_init(elev=30, azim=45)

# Plot full trajectory
trajectory_line, = ax3d.plot(positions[:, 0], positions[:, 1], positions[:, 2])

# Initialize camera marker and arrow
camera_scatter = ax3d.scatter([], [], [])
camera_quiver = ax3d.quiver(0, 0, 0, 0, 0, 0)

# Initialize image display
img_display = ax_img.imshow(images[0])
ax_img.axis('off')

# -- Animation update function --
def update(frame_idx):
    # Update camera position marker
    camera_scatter._offsets3d = (
        positions[frame_idx:frame_idx+1, 0],
        positions[frame_idx:frame_idx+1, 1],
        positions[frame_idx:frame_idx+1, 2]
    )
    # Update camera orientation arrow
    global camera_quiver
    camera_quiver.remove()
    dir_vec = dirs[frame_idx]
    camera_quiver = ax3d.quiver(
        positions[frame_idx, 0], positions[frame_idx, 1], positions[frame_idx, 2],
        dir_vec[0], dir_vec[1], dir_vec[2],
        length=0.1
    )
    # Update displayed image
    img_display.set_data(images[frame_idx])
    return trajectory_line, camera_scatter, camera_quiver, img_display

# -- Create and display animation --
anim = FuncAnimation(fig, update, frames=len(positions), interval=100)
anim.save('animation.gif', writer='pillow')